In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Generate a random regression problem
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [9]:
"L1 norm"
class l1_regularization():
    def __init__(self, alpha):
        self.alpha = alpha
    def __call__(self, w):
        loss = np.sum(np.fabs(w)) 
        return self.alpha*loss
    # The gradient of the L1 norm
    def grad(self, w):
        return self.alpha*np.sign(w)

In [10]:
"L2 norm"
class l2_regularization():
    def __init__(self, alpha):
        self.alpha = alpha
    def __call__(self, w):
        loss = w.T.dot(w)
        return self.alpha*0.5*float(loss)
    # The gradient of L2 norm
    def grad(self, w):
        return self.alpha*w

In [14]:
# Define the LR model
class LinearRegression():
    """Parameters:
    n_iterations: int, the iterations for gradient descent
    learning_rate: float
    regularization: l1_regularization or l2_regularization or None
    gradient: bool, whether to use gradient descent or normal equation. If using regularization, then we can only use gradient descent
    """
    def __init__(self, n_iterations=1000, learning_rate=0.001, regularization=None, gradient=True):
        self.n_iterations = n_iterations
        self.learning_rate = learning_rate
        self.gradient = gradient
        if regularization == None:
            self.regularization = lambda x: 0
            self.regularization.grad = lambda x: 0
        else:
            self.regularization = regularization
        
    def initialize_weights(self, n_features):
        limit = np.sqrt(1/n_features)
        w = np.random.uniform(-limit, limit, (n_features, 1))
        b = 0
        self.w = np.insert(w, 0, b, axis=0)
    
    def fit(self, X, y):
        m_sample, n_features = X.shape
        self.initialize_weights(n_features)
        X = np.insert(X, 0, 1, axis = 1)
        y = np.reshape(y, (m_sample, 1))
        self.training_loss = []
        if self.gradient == True:
            for i in range(self.n_iterations):
                y_pred = X.dot(self.w)
                loss = np.mean(0.5*(y-y_pred)**2)+self.regularization(self.w)
                self.training_loss.append(loss)
                # Calculate the gradient using (y_pred-y)*X
                w_grad = X.T.dot(y_pred-y)+self.regularization.grad(self.w)
                # Update the weights
                self.w = self.w - self.learning_rate*w_grad
        else:
            # Normal equation using (X.T*X).I*X.T*y
            X = np.matrix(X)
            y = np.matrix(y)
            self.w = X.T.dot(X).I.dot(X.T).dot(y)
            
    def predit(self, X):
        X = np.insert(X, 0, 1, axis=1)
        y_pred = np.dot(X, self.w)
        return y_pred    

In [17]:
def main():
    X, y = make_regression(n_samples=100, n_features=1, noise=20)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    n_sample, n_features = np.shape(X)
    
    model = LinearRegression(n_iterations=2000, learning_rate=0.005, regularization=l1_regularization(alpha=0.5), gradient=True)
    model.fit(X_train, y_train)
    
    y_pred = model.predit(X_test)
    y_pred = np.reshape(y_pred, y_test.shape)
    
    mse = mean_squared_error(y_test, y_pred)
    print('Mean squared error:', mse)
    #print(y_pred)
    
if __name__ == '__main__':
    main()

Mean squared error: 441.02963356809175
